In [2]:
import objaverse

objaverse.__version__

'0.1.7'

In [3]:
uids = objaverse.load_uids()
print(f"Found {len(uids)} total objects.")

Found 798759 total objects.


Load Annotations

In [4]:
import random

# This downloads the metadata for all objects (can take a minute the first time)
annotations = objaverse.load_annotations()

# Get a list of UIDs for objects with a permissive CC-BY license
cc_by_uids = [uid for uid, annotation in annotations.items() if annotation["license"] == "by"]
print(f"Found {len(cc_by_uids)} objects with CC-BY license.")

# Let's say you also want to find all the chairs
chair_uids = [uid for uid, annotation in annotations.items() if "chair" in annotation.get("tags", [])]
print(f"Found {len(chair_uids)} chairs.")

100%|██████████| 160/160 [00:33<00:00,  4.84it/s]


Found 704846 objects with CC-BY license.
Found 0 chairs.


### Cell — Inspect annotation structure and find "chair" via tags + LVIS (toggleable)

In [5]:
# Cell — Inspect annotation structure and find "chair" via tags + LVIS (toggleable)

DO_TAGS = True   # use tag-based filtering (Sketchfab tags)
DO_LVIS = True   # use LVIS category lists

# 1) Peek at structure so we don't assume wrong keys/types
any_uid = next(iter(annotations))
print("Sample UID:", any_uid)
print("Top-level keys:", list(annotations[any_uid].keys())[:20])
print("Example tags entry (first 3):", annotations[any_uid].get("tags", [])[:3])

# 2) Helper: safely extract lowercase tag strings from mixed formats
def tag_strings(a):
    out = []
    for t in a.get("tags", []):
        if isinstance(t, dict):
            for k in ("name", "slug"):
                v = t.get(k)
                if isinstance(v, str):
                    out.append(v.lower())
        elif isinstance(t, str):
            out.append(t.lower())
    return set(out)

# 3) Tag-based "chair" (matches 'chair', 'armchair', etc.)
tag_chair_uids = []
if DO_TAGS:
    for uid, a in annotations.items():
        tags = tag_strings(a)
        if any("chair" in s for s in tags):
            tag_chair_uids.append(uid)
print(f"Tag-based 'chair' matches: {len(tag_chair_uids)}")

# 4) LVIS-based "chair" (category lists provided by Objaverse)
lvis_chair_uids = []
if DO_LVIS:
    lvis = objaverse.load_lvis_annotations()
    lvis_lower = {k.lower(): v for k, v in lvis.items()}  # normalize keys
    wanted = {"chair", "armchair", "rocking_chair"}       # expand as needed
    for k in wanted:
        lvis_chair_uids.extend(lvis_lower.get(k, []))
    lvis_chair_uids = list(set(lvis_chair_uids))
    print(f"LVIS-based 'chair' matches: {len(lvis_chair_uids)}")

# 5) Intersect with CC-BY if you computed cc_by_uids earlier
try:
    cc_by_set = set(cc_by_uids)
    print("Within CC-BY — tag:", len(cc_by_set.intersection(tag_chair_uids)),
          "| LVIS:", len(cc_by_set.intersection(lvis_chair_uids)))
except NameError:
    pass

# 6) Final candidate list (union)
CHAIR_UIDS = list(set(tag_chair_uids) | set(lvis_chair_uids))
print(f"Total unique chair candidates (tag ∪ LVIS): {len(CHAIR_UIDS)}")


Sample UID: 94db219c315742909fee67deeeacae15
Top-level keys: ['uri', 'uid', 'name', 'staffpickedAt', 'viewCount', 'likeCount', 'animationCount', 'viewerUrl', 'embedUrl', 'commentCount', 'isDownloadable', 'publishedAt', 'tags', 'categories', 'thumbnails', 'user', 'description', 'faceCount', 'createdAt', 'vertexCount']
Example tags entry (first 3): [{'name': 'game-ready', 'slug': 'game-ready', 'uri': 'https://api.sketchfab.com/v3/tags/game-ready'}, {'name': 'damascus', 'slug': 'damascus', 'uri': 'https://api.sketchfab.com/v3/tags/damascus'}, {'name': 'damascus_steel', 'slug': 'damascus_steel', 'uri': 'https://api.sketchfab.com/v3/tags/damascus_steel'}]
Tag-based 'chair' matches: 3399
LVIS-based 'chair' matches: 573
Within CC-BY — tag: 3185 | LVIS: 548
Total unique chair candidates (tag ∪ LVIS): 3854


In [6]:
from pathlib import Path

uid_to_download = CHAIR_UIDS[0]
print(f"[Step 1] Selected UID: {uid_to_download}")


[Step 1] Selected UID: 07d76d7ce3d14a589eb892beacbec68f


In [7]:
import time

print("[Step 2] Starting download...")
start = time.time()

assets = objaverse.load_objects(
    uids=[uid_to_download],
    download_processes=1
)

print(f"[Step 2] Download complete in {time.time()-start:.2f} seconds.")
print("[Step 2] Downloaded files:", assets[uid_to_download])


[Step 2] Starting download...
Downloaded 1 / 1 objects
[Step 2] Download complete in 2.32 seconds.
[Step 2] Downloaded files: /Users/utkarshsingh/.objaverse/hf-objaverse-v1/glbs/000-118/07d76d7ce3d14a589eb892beacbec68f.glb


In [8]:
print("[Step 3] Searching for mesh file...")

mesh_path = None
candidate_path = assets[uid_to_download]
if isinstance(candidate_path, (str, Path)):
    if str(candidate_path).lower().endswith((".obj", ".stl", ".ply", ".glb", ".gltf", ".off", ".fbx")):
        mesh_path = Path(candidate_path)

if mesh_path:
    print(f"[Step 3] Mesh file found: {mesh_path}")
else:
    print("[Step 3] No mesh file found for this UID.")


[Step 3] Searching for mesh file...
[Step 3] Mesh file found: /Users/utkarshsingh/.objaverse/hf-objaverse-v1/glbs/000-118/07d76d7ce3d14a589eb892beacbec68f.glb


Render Mesh

In [9]:
import trimesh

print("[Step 4] Loading mesh...")
mesh = trimesh.load_mesh(mesh_path)
print("[Step 4] Mesh loaded.")

# Try interactive view
try:
    mesh.show()
except Exception as e:
    print("[Step 4] Interactive view not supported:", e)


[Step 4] Loading mesh...
[Step 4] Mesh loaded.


In [9]:
import open3d as o3d

print("[Step 5] Loading with Open3D...")
o3d_mesh = o3d.io.read_triangle_mesh(str(mesh_path))
o3d_mesh.compute_vertex_normals()

print("[Step 5] Rendering with Open3D...")
o3d.visualization.draw_geometries([o3d_mesh])


[Step 5] Loading with Open3D...
[Step 5] Rendering with Open3D...


### Converting Mesh to PointCloud

In [11]:
def mesh_to_pointcloud(mesh_path, num_points=100_000):
    """
    Load mesh from mesh_path and sample a point cloud of num_points points.
    """
    mesh_o3d = o3d.io.read_triangle_mesh(str(mesh_path))
    mesh_o3d.compute_vertex_normals()
    
    # Sample points uniformly from surface
    pcd = mesh_o3d.sample_points_uniformly(number_of_points=num_points)
    return pcd

# Example usage
NUM_POINTS = 200_000  # Toggle this: 50_000, 100_000, etc.
print(f"[Step 6] Sampling {NUM_POINTS} points from mesh...")
pcd = mesh_to_pointcloud(mesh_path, num_points=NUM_POINTS)
print(f"[Step 6] Point cloud has {len(pcd.points)} points.")


[Step 6] Sampling 200000 points from mesh...
[Step 6] Point cloud has 200000 points.


In [12]:
print("[Step 7] Rendering point cloud...")

# Optional: Set a uniform color for all points
pcd.paint_uniform_color([0.1, 0.6, 0.9])  # light blue

o3d.visualization.draw_geometries([pcd])


[Step 7] Rendering point cloud...


### Converting PointCloud to Mesh

In [14]:
import numpy as np
import open3d as o3d
import time

print("[Reconstruct] Starting Poisson surface reconstruction...")

t0 = time.time()

# 1) Ensure normals (required by Poisson)
print("[Reconstruct] Estimating / orienting normals...")
pcd_est = pcd.voxel_down_sample(voxel_size=0.002) if len(pcd.points) > 300_000 else pcd
pcd_est.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))
pcd_est.orient_normals_consistent_tangent_plane(k=50)

# 2) Poisson reconstruction
# depth controls detail; 9–11 is a good start; increase for more detail (slower)
print("[Reconstruct] Running Poisson...")
mesh_raw, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd_est, depth=10, linear_fit=True
)
densities = np.asarray(densities)

# 3) Crop to the point cloud's bbox (Poisson can create a "balloon" around data)
print("[Reconstruct] Cropping to AABB...")
bbox = pcd.get_axis_aligned_bounding_box()
bbox = bbox.scale(1.02, bbox.get_center())  # small pad
mesh_cropped = mesh_raw.crop(bbox)

# 4) Remove very low-density verts (likely artifacts)
print("[Reconstruct] Removing low-density artifacts...")
densities_cropped = densities[np.asarray(mesh_raw.vertices)[:,0].shape[0] - np.asarray(mesh_cropped.vertices)[:,0].shape[0]:] if len(mesh_cropped.vertices) != len(mesh_raw.vertices) else densities
# If cropping changed vertex order unpredictably, recompute densities by kNN as fallback:
try:
    d = densities_cropped
except Exception:
    # fallback density via distance to nearest point
    p_pts = np.asarray(pcd.points)
    m_pts = np.asarray(mesh_cropped.vertices)
    import scipy.spatial as sps
    kdt = sps.cKDTree(p_pts)
    dists, _ = kdt.query(m_pts, k=8)
    d = 1.0 / (dists.mean(axis=1) + 1e-6)

thresh = np.quantile(d, 0.02)  # drop bottom 2%
mask = d < thresh
mesh_cropped.remove_vertices_by_mask(mask)

# 5) (Optional) Simplify to a manageable size
target_tris = min(150_000, len(mesh_cropped.triangles))
if target_tris > 0:
    print(f"[Reconstruct] Simplifying mesh to ~{target_tris} triangles...")
    mesh_simplified = mesh_cropped.simplify_quadric_decimation(target_number_of_triangles=target_tris)
else:
    mesh_simplified = mesh_cropped

# 6) Final cleanups
mesh_simplified.remove_unreferenced_vertices()
mesh_simplified.remove_degenerate_triangles()
mesh_simplified.remove_duplicated_vertices()
mesh_simplified.remove_duplicated_triangles()

print(f"[Reconstruct] Done in {time.time()-t0:.2f}s | V:{len(mesh_simplified.vertices)} F:{len(mesh_simplified.triangles)}")

# Keep result for next cell
mesh_recon = mesh_simplified


[Reconstruct] Starting Poisson surface reconstruction...
[Reconstruct] Estimating / orienting normals...
[Reconstruct] Running Poisson...
[Reconstruct] Cropping to AABB...
[Reconstruct] Removing low-density artifacts...
[Reconstruct] Simplifying mesh to ~150000 triangles...
[Reconstruct] Done in 18.90s | V:84014 F:149999


In [15]:
print("[Render] Showing reconstructed mesh...")
o3d.visualization.draw_geometries([mesh_recon])


[Render] Showing reconstructed mesh...
